In [ ]:
# We have to use it this way until the next PyNN release
import spynnaker.pyNN as pynn

# Set up some constants
n_neurons = 100
n_populations = 10
weights = 0.5
delays = 17.0
simtime = 1000

# Set up pyNN with a timestep of 1.0ms
pynn.setup(timestep=1.0)

# Set up a Population to start the synfire chain
stimulus = pynn.Population(1, pynn.SpikeSourceArray(spike_times=[0]), label='stimulus')

# Set up the Synfire chain populations
chain_pops = [
    pynn.Population(n_neurons, pynn.IF_curr_exp(), label='chain_{}'.format(i))
    for i in range(n_populations)
]

# Record the spikes in the Populations so we can see them at the end!
for pop in chain_pops:
    pop.record("spikes")
    
# Connect them together
connector = pynn.FixedNumberPreConnector(10)
for i in range(n_populations):
    pynn.Projection(
        chain_pops[i], chain_pops[(i + 1) % n_populations], 
        connector, synapse_type=pynn.StaticSynapse(weight=weights, delay=delays))
    
# Connect the simulus to start the chain
pynn.Projection(stimulus, chain_pops[0], pynn.AllToAllConnector(), synapse_type=pynn.StaticSynapse(weight=5.0))

# Run the Simulation
pynn.run(simtime)

# Extract the data
neo_data = [pop.get_data("spikes") for pop in chain_pops]

spike_trains = [neo.segments[0].spiketrains for neo in neo_data]
offset = 0
for spike_train_list in spike_trains:
    for spike_train in spike_train_list:
        spike_train.annotations["source_index"] += offset
    offset += n_neurons

# End the simulation
pynn.end()

In [ ]:
# Plot the results
import matplotlib.pyplot as plt
import numpy
import pyNN.utility.plotting as plotting
%matplotlib inline

colours = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

plt.figure()
plt.xlim(0, simtime)
plt.ylim(0, n_neurons * n_populations)
offset = 0
for i, pop_spikes in enumerate(spike_trains):
    colour = colours[i % len(colours)]
    for spike_train in pop_spikes:
        neuron_id = spike_train.annotations['source_index'] + offset
        plt.plot(spike_train, numpy.ones_like(spike_train) * neuron_id, ".", color=colour)
    offset += len(spike_train)
plt.show()